In this notebook, we reproject all point clouds that have been clipped to lidar plots from the Washington State Plane coordinate reference system over to UTM. The WA DNR Lidar Portal stores all its lidar point clouds in this CRS, but we will be moving everything over to UTM.

In [1]:
import geopandas as gpd
import pandas as pd
import os
import glob
import subprocess
import dask
from dask.distributed import Client, LocalCluster, progress

In [2]:
PLOTS = '../data/processed/blm_usfs_wadnr_plot_footprints.shp'
plots = gpd.read_file(PLOTS)
plots.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5089 entries, 0 to 5088
Data columns (total 10 columns):
comments     496 non-null object
lat          5089 non-null float64
lon          5089 non-null float64
meas_date    3866 non-null object
orig_id      5089 non-null object
plot_id      3866 non-null object
source       5089 non-null object
meas_yr      5089 non-null int64
uuid         5089 non-null object
geometry     5089 non-null object
dtypes: float64(2), int64(1), object(7)
memory usage: 397.7+ KB


In [3]:
plots.crs

{'init': 'epsg:4326'}

In [4]:
UTM_10 = '../data/external/utm_zone10_epsg4326.shp'
UTM_11 = '../data/external/utm_zone11_epsg4326.shp'
utm_10 = gpd.read_file(UTM_10)
utm_11 = gpd.read_file(UTM_11)
utm_10.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
SWLON         1 non-null object
SWLAT         1 non-null object
HEMISPHERE    1 non-null object
ZONE          1 non-null object
CM            1 non-null object
Zone_Hemi     1 non-null object
geometry      1 non-null object
dtypes: object(7)
memory usage: 136.0+ bytes


In [5]:
utm_10.crs

{'init': 'epsg:4326'}

In [6]:
utm10_plots = gpd.sjoin(plots, utm_10)
utm11_plots = gpd.sjoin(plots, utm_11)

utm_lookup = pd.concat((utm10_plots, utm11_plots), 
                       axis=0, 
                       ignore_index=True)[['uuid', 'ZONE']].set_index('uuid')
utm_lookup['epsg'] = utm_lookup.ZONE.apply(lambda x: 6339 if x == '10' else 6340)
utm_lookup.head()

,ZONE,epsg
uuid,,
d7c01e3a-38e0-4bc2-a69c-7d5a204e2663,10,6339
c16be14e-f913-4516-9c4b-078b3d71371d,10,6339
b4e059b8-6f08-4d33-8c3c-2b0410d2a226,10,6339
e3b77390-f724-4a58-ad90-0b97cb138ce8,10,6339
5588b367-b5dc-4b23-9af3-bcf0d6e2a571,10,6339


In [7]:
len(utm_lookup)

5089

In [8]:
@dask.delayed
def reproject(infile, odir):
    basename = os.path.basename(infile)
    outfile = os.path.join(odir, basename)
    
    uuid = basename.split('_')[0]
    target_epsg = utm_lookup.loc[uuid]['epsg']
    
    proc = subprocess.run(['wine', '/storage/lidar/LAStools/bin/las2las.exe',
                           '-i', infile,
                           '-o', outfile,
                           '-target_epsg', str(target_epsg)],
                          stderr=subprocess.PIPE,
                          stdout=subprocess.PIPE)
    return proc

In [9]:
USFS_HA = glob.glob('../data/raw/lidar/dnr_portal/dnr_portal_tiles/usfs_plots/hectare_clips_epsg2927/*.laz')
USFS_PLOT = glob.glob('../data/raw/lidar/dnr_portal/dnr_portal_tiles/usfs_plots/plot_clips_epsg2927/*.laz')
DNR_HA = glob.glob('../data/raw/lidar/dnr_portal/dnrplot_clips/hectare_clips_epsg2927/*.laz')
DNR_PLOT = glob.glob('../data/raw/lidar/dnr_portal/dnrplot_clips/plot_clips_epsg2927/*.laz')

In [10]:
cluster=LocalCluster(scheduler_port=7001, diagnostics_port=7002)
c = Client(cluster)

In [36]:
jobs_to_do = []
PLOT_SETS = [USFS_HA, USFS_PLOT, DNR_HA, DNR_PLOT]
ODIRS = ['../data/interim/lidar/plot_clips/dnr_portal/usfs_plots/hectare_clips/',
         '../data/interim/lidar/plot_clips/dnr_portal/usfs_plots/plot_clips/',
         '../data/interim/lidar/plot_clips/dnr_portal/dnr_plots/hectare_clips/',
         '../data/interim/lidar/plot_clips/dnr_portal/dnr_plots/plot_clips/']

for plot_set, odir in zip(PLOT_SETS, ODIRS):
    jobs_to_do += [reproject(p, odir=odir) for p in plot_set]

In [ ]:
res = c.persist(jobs_to_do)
progress(res)

In [11]:
@dask.delayed
def get_boundary(infile, odir):
    proc = subprocess.run(['wine', '/storage/lidar/LAStools/bin/lasboundary.exe',
                           '-i', infile,
                           '-odir', odir,
                           '-oshp',
                           '-labels'], 
                          stderr=subprocess.PIPE,
                          stdout=subprocess.PIPE)
#     print(outfile)
    return proc

In [12]:
USFS_HA = glob.glob('../data/interim/lidar/plot_clips/dnr_portal/usfs_plots/hectare_clips/*.laz')
USFS_PLOT = glob.glob('../data/interim/lidar/plot_clips/dnr_portal/usfs_plots/plot_clips/*.laz')
DNR_HA = glob.glob('../data/interim/lidar/plot_clips/dnr_portal/dnr_plots/hectare_clips/*.laz')
DNR_PLOT = glob.glob('../data/interim/lidar/plot_clips/dnr_portal/dnr_plots/plot_clips/*.laz')

jobs_to_do = []
PLOT_SETS = [USFS_HA, USFS_PLOT, DNR_HA, DNR_PLOT]
ODIRS = ['../data/interim/lidar/plot_clips/dnr_portal/usfs_plots/hectare_clips/',
         '../data/interim/lidar/plot_clips/dnr_portal/usfs_plots/plot_clips/',
         '../data/interim/lidar/plot_clips/dnr_portal/dnr_plots/hectare_clips/',
         '../data/interim/lidar/plot_clips/dnr_portal/dnr_plots/plot_clips/']

for plot_set, odir in zip(PLOT_SETS, ODIRS):
    jobs_to_do += [get_boundary(p, odir=odir) for p in plot_set]

In [13]:
res = c.persist(jobs_to_do)
progress(res)

VBox()

In [38]:
# c.close()
# cluster.close()